In [2]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using StatsBase
using Profile

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [3]:
dataset = "contact-primary-school"
kmax_ = 6

H, Z = read_hypergraph_data(dataset,kmax_)

kmax = maximum(keys(H.E))
kmin = minimum(keys(H.E))

n = length(H.D)

# full, partition-based intensity function

function ω_1(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

Ω_1 = partitionIntensityFunction(ω_1, kmax);

# ---

# sum of exterior
function ω_2(p,α)
    k = p[2]
    δ = p[1]
    return ((1+δ)*n)^α[k] / (1 + α[k + kmax])
end

Ω_2 = sumOfExteriorDegreesIntensityFunction(ω_2, kmax);


# sum of exterior
function ω_3(p,α)
    k = p[2]
    δ = p[1]
    return ((1+δ)*n)^α[k] / (1 + α[k + kmax])
end

Ω_3 = allOrNothingIntensityFunction(ω_3, kmax);

In [4]:
Z_dyadic = CliqueExpansionModularity(H);
α = zeros(2*kmax);

In [9]:
α, f = learnParameters(H, Z_dyadic, Ω_3, α; xtol_abs = 1e-6)

([-3.7916786492803087, -0.8163874991460648, -2.7646155113883406, -5.795283685851312, -8.795783455848424, 0.0017717961666377953, 1215.690958789531, 1440.3548676158134, 34.996295125448626, 2.1364582194308683], 228815.91980750754)

In [10]:
Profile.clear()
@profile Z = HypergraphModularity.HyperLouvain(H, kmax, Ω_3; α=α, verbose = true);

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
Louvain Iteration 8
Louvain Iteration 9
Louvain Iteration 10
Louvain Iteration 11
Louvain Iteration 12
Louvain Iteration 13


In [13]:
Profile.print(sortedby = :count, mincount = 500)

Overhead ╎ [+additional indent] Count File:Line; Function
    ╎8071  @Base/task.jl:358; (::IJulia.var"#15#18")()
    ╎ 8071  ...ia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
    ╎  8071  @Base/essentials.jl:711; invokelatest
    ╎   8071  @Base/essentials.jl:712; #invokelatest#1
    ╎    8071  ...execute_request.jl:67; execute_request(::ZMQ.Socket...
    ╎     8071  ...oftGlobalScope.jl:218; softscope_include_string(::...
    ╎    ╎ 8071  @Base/boot.jl:331; eval
    ╎    ╎  8071  ...rgraph_louvain.jl:2; (::HypergraphModularity.var...
   1╎    ╎   8071  ...rgraph_louvain.jl:2; HyperLouvain##kw
  25╎    ╎    1143  ...raph_louvain.jl:74; HyperLouvain(::hypergraph...
    ╎    ╎     1118  @Base/reduce.jl:503; sum(::Base.Generator{Bas...
    ╎    ╎    ╎ 1118  @Base/reduce.jl:486; sum
    ╎    ╎    ╎  1118  @Base/reduce.jl:283; mapreduce
    ╎    ╎    ╎   1118  @Base/reduce.jl:283; #mapreduce#193
    ╎    ╎    ╎    1118  @Base/reduce.jl:157; mapfoldl
    ╎    ╎    ╎     1118  @Base/reduce.jl